In [41]:
# %pip install shutils pandas librosa soundfile pandas ffmpeg-python

In [42]:
import sys

sys.path.append("../")
import shutil
import os
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
# from app.main import process_audio_file_task
from lib.audio import splice_audio_stream

In [43]:
NUM_WORKERS = 1

# Set paths
PARENT_DIR = '/Users/paul.mutemi/Documents/ccai'
MANIFEST = f"{PARENT_DIR}/tmp/csv"
ORIGIN_PATH = f"{PARENT_DIR}/tmp"
DESTINATION_DIR = f"{PARENT_DIR}/tmp/segments"
CSV_FILE = f"{MANIFEST}/segments.csv"



In [44]:
if os.path.exists(DESTINATION_DIR):
    shutil.rmtree(DESTINATION_DIR, ignore_errors=True)
    print(f"OK => {DESTINATION_DIR} has been removed")
    
if os.path.exists(MANIFEST):
    shutil.rmtree(MANIFEST, ignore_errors=True)
    print(f"OK => {MANIFEST} has been removed")
    
if not os.path.exists(DESTINATION_DIR):
    print("OK => Creating destination_folder")
    os.makedirs(DESTINATION_DIR)
    print(f"Ok => Created {DESTINATION_DIR}")
    os.makedirs(MANIFEST)
    print(f"Ok => Created {MANIFEST}")    

OK => /Users/paul.mutemi/Documents/ccai/tmp/segments has been removed
OK => /Users/paul.mutemi/Documents/ccai/tmp/csv has been removed
OK => Creating destination_folder
Ok => Created /Users/paul.mutemi/Documents/ccai/tmp/segments
Ok => Created /Users/paul.mutemi/Documents/ccai/tmp/csv


In [45]:
print("Segmenting audio files!")

all_file_names = [
    os.path.join(ORIGIN_PATH, f)
    for f in os.listdir(ORIGIN_PATH)
    if os.path.splitext(f)[1] in [".mp3", ".wav", ".m4a", ".aac"]
]
data = []

with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = [
        executor.submit(splice_audio_stream, file_path, DESTINATION_DIR)
        for file_path in all_file_names
    ]
    for future in as_completed(futures):
        data.extend(future.result())

df = pd.DataFrame(data)
df.to_csv(CSV_FILE, index=False)

print(f"Saved dataframe in: {CSV_FILE}")

Segmenting audio files!
Saved dataframe in: /Users/paul.mutemi/Documents/ccai/tmp/csv/segments.csv
